In [1]:
from src.utils.generator.Generator import Generator as Generator
from src.utils.shocker.ShockerDebt import ShockerDebt as ShockerDebt
from src.utils.shocker.ShockerCapital import ShockerCapital as ShockerCapital
from src.utils.shocker.ShockerWagesEmployment import ShockerWagesEmployment as ShockerWagesEmployment

import os
import shutil
from distutils.dir_util import copy_tree

# Parametros 1

In [2]:
root_experiments = '/home/pedro/Desktop/Proyectos/multi-formalism-translator/notebooks-dev/results-experiments/'

experiment_name = "sample_experiment"
experiment_directory = root_experiments + '/' + experiment_name

src_folder = root_experiments + experiment_name + '/src/'
cnt_parallel = 8

In [3]:
templates_dir = '/home/pedro/Desktop/Proyectos/multi-formalism-translator/test/data/templates-experimentos'
shocker_templates_folder = templates_dir + '/shockers'

In [4]:
# %%sh -s "$experiment_directory"
# mkdir "$1"

In [5]:
experiment_base_src_dir = '/home/pedro/Desktop/Proyectos/multi-formalism-translator/test/data/modelos/cell/opinion-goodwin-minsky/src'

copy_tree(experiment_base_src_dir, src_folder)
print ("Copied OK")

Copied OK


### TODO: criterio de 'shockExterno' => que pasa si shockeas a un Influyente?

In [17]:
simulation_stop_time = "00:01:00:00:000"
cell_devs_width = 10
cell_devs_height = 10

# Shockers
threshold_WR = 0.5
threshold_ER = 0.5

solo_una_vez = False
shockers = [
    # (type_, name_, limit, solo_un_shock, n_outports, n_outports_to_shock, nombre_shock)
    ShockerDebt(shocker_templates_folder, 7, 'shockerDebt1', 1000, solo_una_vez, cell_devs_width * cell_devs_height, 30, "shockExterno"),
    ShockerDebt(shocker_templates_folder, 6, 'shockerDebt2', 3500, solo_una_vez, cell_devs_width * cell_devs_height, 30, "shockExterno"),
    ShockerCapital(shocker_templates_folder, 5, 'shockerCapital1', 500, solo_una_vez, cell_devs_width * cell_devs_height, 30, "shockExterno"),
    
    # (type_, name_, thresholdWR, thresholdER, negativeshock, positiveshock, 
    # solo_un_shock, n_outports, n_outports_to_shock, nombre_shock)
    ShockerWagesEmployment(shocker_templates_folder, 7, 'shockerWRER2', threshold_WR, threshold_ER, 5, 7, solo_una_vez,
                           cell_devs_width * cell_devs_height, 30, "shockExterno")
]

# POR LAS DUDAS: limito la cantidad de threads cuando el modelo es el COMBINADO (no-standalone)
# TODO: VER DE CORRER EN PARALELO CUANDO QUEREMOS CON Y SIN SHOCK
GENERAR_CON_Y_SIN_SHOCK = True
if len(shockers) > 0:
    cnt_parallel = 4

for shocker in shockers:
    shocker.generate_shocker(src_folder)
cell_devs_delay_value = 100
cell_devs_shock_rules = {
    'shockExterno': [
        '{ (0,0,0) - #macro(qshock) * #macro(delta) } 0 { portValue(thisPort) = 5 and (0,0,0) - #macro(qshock) * #macro(delta) >= -3}',
        '{ -3 } 0 { portValue(thisPort) = 5 and (0,0,0) - #macro(qshock) * #macro(delta) < -3}',
        
        '{ (0,0,0) } 0 { portValue(thisPort) = 6 and (0,0,0) = 0}',
        
        '{ (0,0,0) + #macro(qshock) * #macro(delta) } 0 { portValue(thisPort) = 6 and (0,0,0) < 0 and (0,0,0) + #macro(qshock) * #macro(delta) <= 3}',
        '{ 3 } 0 { portValue(thisPort) = 6 and (0,0,0) < 0 and (0,0,0) + #macro(qshock) * #macro(delta) > 3}',
        
        '{ (0,0,0) - #macro(qshock) * #macro(delta) } 0 { portValue(thisPort) = 6 and (0,0,0) > 0 and (0,0,0) - #macro(qshock) * #macro(delta) >= -3}',
        '{ -3 } 0 { portValue(thisPort) = 6 and (0,0,0) > 0 and (0,0,0) - #macro(qshock) * #macro(delta) < -3}',
        
        '{ (0,0,0) + #macro(qshock) * #macro(delta) } 0 { portValue(thisPort) = 7 and (0,0,0) + #macro(qshock) * #macro(delta) <= 3}',
        '{ 3 } 0 { portValue(thisPort) = 7 and (0,0,0) + #macro(qshock) * #macro(delta) > 3}',
        
        '{ (0,0,0) } 0 { t }'
    ]
}

n_initial_conditions = 10

# InitialConditions: [%extreme_neg,%negative, %neutral, %positive, %extreme_pos]
# TODO: HACER QUE DEPENDA DE 'long' para cada valor de la MACRO
# pctgs = []
# for p0 in [10,20,30,40,50,60,70,80,90]:
#     for pi in range(10,100-p0+1,10):
#         m = (100 - pi - p0)/2
#         k = [pi/2, m, float(p0), m, pi/2]
#         assert(sum(k) == 100)
#         k2 = list(map(lambda x: x/100, k))
#         assert(round(sum(k2),5) == 1)
#         pctgs.append(k2)  
# initial_cond_percentages = pctgs
# initial_cond_percentages = [[0.1, 0.05, 0.7, 0.05, 0.1]]
initial_cond_percentages = [
    #[0.05,0,0.9,0,0.05],
    [0.1 ,0.05,0.7,0.05,0.1],
    #[0.2,0  ,0.6,0,0.2]
]

# Explicacion parametros:
# long: - determina el limite entre Partidario / Influyente
#       - [-1,1] => Indefinido | [1,long] => Partidario | [long, 3] => Influyente
# q: - si yo soy Partidiario y el otro es Influyente del otro bando, me acerco un q*delta.
#    - si yo soy Indefinido y el otro es Influyente, me acerco un q*delta
#
# k: - si yo soy Indefinido y el otro es Partidiario , me acerco un k*delta.
# 
# ===> debe pasar : q > k
#
macros_list = [
    {"long": 2.8, "q": 2, "delta": 0.1, "k": 1.5, "qshock": 13},
    {"long": 2.8, "q": 2, "delta": 0.2, "k": 1.5, "qshock": 13}
]
input_parameters_list = [
    {
        'Betaa' : 0.015, 
        'rateInterestOnLoans' : 0.04, 
        'Alphaa' : 0.025, 
        'employmentRateStable' : 10,
        'deltaKReal' : 0.05, 
        'velocityOfMoney' : 3,
        'piZ' : 0.04,
        'employmentRateZero' : 0.6,
        'piS' : 10,
        
        'ThresholdWR': 0.8,
        'ThresholdER': 0.8
    }
]

template-shocker-wages-employmentrate.cpp


# Genero experimentos

In [18]:
g = Generator(templates_dir)
g.generate_reg_file(src_folder + 'reg.cpp', shockers)
for icp in initial_cond_percentages:
    for n in range(n_initial_conditions):
        name_valfiles = experiment_directory + "/valfiles/"
        if not os.path.exists(name_valfiles):
            os.mkdir(name_valfiles)
        name = name_valfiles + "_".join(list(map(lambda x: str(x), icp))) + "/"
        if not os.path.exists(name):
            os.mkdir(name)
        name = name + "valfile_" + str(n)
        g.generate_valfile(name, icp, cell_devs_width, cell_devs_height)

#####################################################################  
g.generate_goodwin_minsky_ma_file(
    experiment_directory + "/mafile.ma",
    shockers,
    cell_devs_height, 
    cell_devs_width, 
    cell_devs_delay_value,
    cell_devs_shock_rules
)


for i, macro in enumerate(macros_list):
    if not os.path.exists(experiment_directory + '/macros'):
        os.mkdir(experiment_directory + '/macros')
    macros_filename = experiment_directory + '/macros/' + "macro_" + str(i)
    g.generate_macros(macros_filename, macro["long"], macro["q"], macro["delta"], macro["k"], macro["qshock"])
for i, ip in enumerate(input_parameters_list):
    if not os.path.exists(experiment_directory + '/events/'):
        os.mkdir(experiment_directory + '/events/')
    g.generate_ev_file(experiment_directory + '/events/events_' + str(i) + '.ev', ip)

# Correr los experimentos

In [23]:
%%sh -s "$experiment_directory"
# rm -r "$1/results"
mkdir "$1/results"

In [24]:
# EVENTOS
for fn in os.listdir(experiment_directory + '/events'):
    for valfile_dir in os.listdir(experiment_directory + '/valfiles'):
        # VALORES INICIALES DE CELDAS
        for valfile in os.listdir(experiment_directory+'/valfiles/'+valfile_dir):
            exp_run_dir = experiment_directory + "/results/" + valfile_dir + "/"
            print(exp_run_dir)
            if not os.path.exists(exp_run_dir):
                os.mkdir(exp_run_dir)
            exp_run_dir = exp_run_dir + valfile + "/"
            if not os.path.exists(exp_run_dir):
                os.mkdir(exp_run_dir)
            # VALORES DE MACROS
            for macro in os.listdir(experiment_directory+'/macros'):
                exp_run_dir_final = exp_run_dir + macro + "/"
                if not os.path.exists(exp_run_dir_final):
                    os.mkdir(exp_run_dir_final)
                shutil.copy(experiment_directory+'/macros/'+macro, exp_run_dir_final+'macros.inc')
                shutil.copy(experiment_directory+'/events/'+fn, exp_run_dir_final + 'events.ev')
                shutil.copy(experiment_directory+'/valfiles/'+valfile_dir+'/'+valfile, exp_run_dir_final+'valfile.val')
                shutil.copy(experiment_directory+'/mafile.ma', exp_run_dir_final+'mafile.ma')
                shutil.copy(templates_dir + '/conviccion.pal', exp_run_dir_final+'conviccion.pal')
                g.generate_opinion_standalone_ma_file(
                    exp_run_dir_final+'opinion_standalone.ma', 
                    cell_devs_width, cell_devs_height
                )

/home/pedro/Desktop/Proyectos/multi-formalism-translator/notebooks-dev/results-experiments//sample_experiment/results/0.2_0_0.6_0_0.2/
/home/pedro/Desktop/Proyectos/multi-formalism-translator/notebooks-dev/results-experiments//sample_experiment/results/0.2_0_0.6_0_0.2/
/home/pedro/Desktop/Proyectos/multi-formalism-translator/notebooks-dev/results-experiments//sample_experiment/results/0.2_0_0.6_0_0.2/
/home/pedro/Desktop/Proyectos/multi-formalism-translator/notebooks-dev/results-experiments//sample_experiment/results/0.2_0_0.6_0_0.2/
/home/pedro/Desktop/Proyectos/multi-formalism-translator/notebooks-dev/results-experiments//sample_experiment/results/0.2_0_0.6_0_0.2/
/home/pedro/Desktop/Proyectos/multi-formalism-translator/notebooks-dev/results-experiments//sample_experiment/results/0.2_0_0.6_0_0.2/
/home/pedro/Desktop/Proyectos/multi-formalism-translator/notebooks-dev/results-experiments//sample_experiment/results/0.2_0_0.6_0_0.2/
/home/pedro/Desktop/Proyectos/multi-formalism-translato

In [25]:
cmds_directory = experiment_directory + "/cmds"
if not os.path.exists(cmds_directory):
    os.mkdir(cmds_directory)

cmds1 = []
cmds2 = []
for props_dir in os.listdir(experiment_directory + '/results'):
    for valfile_dir in os.listdir(experiment_directory+'/results/'+props_dir):
        for macros_dir in os.listdir(experiment_directory+'/results/'+props_dir+'/'+valfile_dir):
            cmd1 = "cd " + 'results/'+props_dir+'/'+valfile_dir+'/'+macros_dir

            # CON SHOCK
            if len(shockers) > 0:
                cmd2_1 = "../../../../src/bin/cd++ -mmafile.ma -eevents.ev -t" + simulation_stop_time + " -llog"
                cmd3_1 = "cp log" + str(45 + len(shockers)) + " opinion_log && rm log*"
                cmd = cmd1 + " && " + cmd2_1 + " && " + cmd3_1 + "\n"
                cmds1.append(cmd)

            # SIN SHOCK
            if len(shockers) == 0 or GENERAR_CON_Y_SIN_SHOCK:
                cmd2_2 = "../../../../src/bin/cd++ -mopinion_standalone.ma -t" + simulation_stop_time + " -llog"
                cmd3_2 = "cp log01 opinion_log_standalone && rm log*"
                cmd2 = cmd1 + " && " + cmd2_2 + " && " + cmd3_2 + "\n"
                cmds2.append(cmd2)



with open (cmds_directory + "/commands_make.txt", "w+") as f:
    f.write("cd src && rm -rf bin && make\n")

n_cmd = 0
filenames_cmds = set()
write_read = ["w+", "a"]
for n_cmds, cmds in enumerate([cmds1, cmds2]):
    for i, cmd in enumerate(cmds):
        if i % cnt_parallel == 0:
            n_cmd += 1

        filename_cmd = cmds_directory + "/commands_" + str(n_cmds) + "_" + str(n_cmd) + ".txt"
        filenames_cmds.add("cmds/commands_" + str(n_cmds) + "_"  + str(n_cmd) + ".txt")

        if i%cnt_parallel==0:
            with open(filename_cmd, "w+") as f:
                f.write(cmd)
        else:
            with open(filename_cmd, "a") as f:
                f.write(cmd)

with open (experiment_directory +"/run.sh", "w+") as f:
    f.write("time parallel :::: cmds/commands_make.txt\n")
    for filename in filenames_cmds:
        f.write("time parallel :::: " + filename + "\n")

if len(shockers) == 0:
    print ("NO HAY SHOCKERS. CORRO EL MODELO OPINION_STANDALONE!!!")


In [26]:
# Borrar archivos utilizados para generar casos de test
shutil.rmtree(experiment_directory+'/macros/')
shutil.rmtree(experiment_directory+'/events/')
shutil.rmtree(experiment_directory+'/valfiles/')